In [5]:
import cv2
import numpy as np
import os

In [6]:
video_folder = "/home/persie/Videos/mokap/"
session_id = os.listdir(video_folder)
video_file = []
for folder in session_id:
    files = os.listdir(os.path.join(video_folder, folder))
    for file in files:
        video_file.append(os.path.join(video_folder, folder, file))


In [7]:
cap = cv2.VideoCapture(video_file[33])
cap.set(cv2.CAP_PROP_POS_FRAMES, 134)
ret, frame = cap.read()
clear_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [58]:
cap = cv2.VideoCapture(video_file[33])
cap.set(cv2.CAP_PROP_POS_FRAMES, 1507)
ret, frame = cap.read()
object_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [9]:
def draw_circle(event,x,y,flags,param):
    #Because of the use of global variables, they must be defined before using them, outside of any functions
    global radius, text_image, mask  
    if flags == cv2.EVENT_FLAG_SHIFTKEY:
            cv2.circle(mask,(x,y),radius,(0,0,0),-1)    
    if flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(mask,(x,y),radius,(255,255,255),-1)      
        
    text_image = cv2.rectangle(text_image, (0, 70), (35, 40), (255, 255, 255), -1)    
    text_image = cv2.putText(text_image, str(radius), (0, 65), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(0, 0, 255), thickness=2)         
        

In [15]:
#Select frame with object in
cap = cv2.VideoCapture(video_file[33])
cap.set(cv2.CAP_PROP_POS_FRAMES, 1507)
ret, frame = cap.read()
object_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Run the mask creation code
text_image = object_frame
mask = np.zeros_like(object_frame, np.uint8)
# Set the initial circle radius
radius = 50
line_1 = "Click with the left mouse button to add, double click to remove"
line_2 = "Up = radius + 5,"
line_3 = "Down = radius - 5"
text_image = cv2.putText(text_image, line_1, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
text_image = cv2.putText(text_image, line_2, (10, 22), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
text_image = cv2.putText(text_image, line_3, (10, 34), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
combined_image = cv2.bitwise_or(text_image, mask)
cv2.namedWindow('Mask On Frame')
cv2.imshow('Mask On Frame', combined_image)
cv2.setMouseCallback('Mask On Frame',draw_circle)


while(1):
    combined_image = cv2.bitwise_or(text_image, mask)
    cv2.imshow('Mask On Frame',combined_image)
    cv2.imshow('Mask',mask)
    k = cv2.waitKey(20) & 0xFF
    if k == ord("q"):
        break
    if k == 82:
        #upkey
        radius = radius + 5
    if k == 84:
        #downkey
        radius = radius - 5

cap.release()
cv2.destroyAllWindows()

In [16]:
object_mask = mask

In [22]:
#Find initial moment
contours, hierarchy = cv2.findContours(object_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# Draw contours:
cv2.drawContours(object_frame, contours, 0, (0, 255, 0), 2)

# Calculate image moments of the detected contour
M = cv2.moments(contours[0])

# Print center (debugging):
print("center X : '{}'".format(round(M['m10'] / M['m00'])))
print("center Y : '{}'".format(round(M['m01'] / M['m00'])))

# Draw a circle based centered at centroid coordinates
cv2.circle(object_frame, (round(M['m10'] / M['m00']), round(M['m01'] / M['m00'])), 5, (0, 255, 0), -1)

# Show image:
cv2.imshow("outline contour & centroid", text_image)

# Wait until a key is pressed:
cv2.waitKey(0)

# Destroy all created windows:
cv2.destroyAllWindows()

center X : '751'
center Y : '542'


In [109]:
#Attempt to refine the mask
thresh = cv2.mean(object_frame, mask=cv2.bitwise_not(object_mask))[0]
average_bg = cv2.bitwise_and(cv2.bitwise_not(object_mask), thresh)
masked_seed = cv2.bitwise_and(object_frame, object_frame, mask=object_mask)
thresh_bg = np.ones_like(object_frame)*np.ceil(thresh).astype(np.uint8)
average_image = cv2.bitwise_xor(masked_seed, average_bg)

# global thresholding
ret1,th1 = cv2.threshold(object_frame,127,255,cv2.THRESH_BINARY)
# Otsu's thresholding
ret2,th2 = cv2.threshold(average_image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(average_image,(7,7),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imshow("blended", average_image)
cv2.imshow("th1", th1)
cv2.imshow("th2", th2)
cv2.imshow("th3", th3)

# Wait until a key is pressed:
cv2.waitKey(0)

# Destroy all created windows:
cv2.destroyAllWindows()


In [176]:
src = th3.copy()
import random as rng
def thresh_callback(val):
    global thresh
    thresh = val
    

# Create Window
source_window = 'Source'
cv2.namedWindow(source_window, cv2.WINDOW_NORMAL)
max_thresh = 255

thresh = 25 # initial threshold
cv2.createTrackbar('Canny thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)
cv2.imshow('Source', src)
color = []
for n in range(0,20):
    color.append((rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)))


    blur_fg = cv2.blur(src, (3,3))
    # Detect edges using Canny
    canny_output = cv2.Canny(blur_fg, thresh, thresh * 2)
    # Find contours
    contours,_ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    edges = np.empty((0,2), dtype=np.uint8)
    
    #Find the overall contour
    for i in range(len(contours)):
        if len(contours[i]) > 1:
            edges = np.concatenate([edges, *contours[i]])
    
    convex_hull = cv2.convexHull(edges)
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    new_edge = object_frame.copy()
    cv2.drawContours(drawing, [convex_hull], -1, color[0], 1)
    cv2.drawContours(new_edge, [convex_hull], -1, color[0], 1)
    
    
while True:
    # Show in a window
    cv2.imshow('Contours', drawing)
    cv2.imshow('New', new_edge)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()





In [20]:
# generate initial corners of detected object
# set limit, minimum distance in pixels and quality of object corner to be tracked
parameters_shitomasi = dict(maxCorners=100, qualityLevel=0.3, minDistance=7)
# set min size of tracked object, e.g. 15x15px
parameter_lucas_kanade = dict(winSize=(40, 40), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))



cap = cv2.VideoCapture(video_file[33])
# Read the first frame

cap.set(cv2.CAP_PROP_POS_FRAMES, 2934)
ret, frame = cap.read()
frame_gray_init = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Use Shi-Tomasi to detect object corners / edges from initial frame
edges = cv2.goodFeaturesToTrack(frame_gray_init, mask = object_mask, **parameters_shitomasi)


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert current frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Frame', frame_gray)


    # create a black canvas the size of the initial frame
    canvas = np.zeros_like(frame)
    # create random colours for visualization for all 100 max corners for RGB channels
    colours = np.random.randint(0, 255, (100, 3))
    
    # update object corners by comparing with found edges in initial frame
    update_edges, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray, edges, None,
                                                         **parameter_lucas_kanade)
    # only update edges if algorithm successfully tracked
    new_edges = update_edges[status == 1]
    # to calculate directional flow we need to compare with previous position
    old_edges = edges[status == 1]

    for i, (new, old) in enumerate(zip(new_edges, old_edges)):
        a, b = new.ravel()
        c, d = old.ravel()

        # draw line between old and new corner point with random colour
        mask = cv2.line(canvas, (int(a), int(b)), (int(c), int(d)), colours[i].tolist(), 2)
        # draw circle around new position
        frame = cv2.circle(frame, (int(a), int(b)), 5, colours[i].tolist(), -1)

    result = cv2.add(frame, mask)
    cv2.imshow('Optical Flow (sparse)', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # overwrite initial frame with current before restarting the loop
    frame_gray_init = frame_gray.copy()
    # update to new edges before restarting the loop
    edges = new_edges.reshape(-1, 1, 2)


cap.release()
cv2.destroyAllWindows()


In [19]:
cap.release()
cv2.destroyAllWindows()
